<a href="https://colab.research.google.com/github/sevaroy/Prophet-playbook/blob/main/Prophet_ETH.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tqdm
!pip install pandas
!pip install prophet
!pip install yfinance
!pip install plotly
!pip install datetime
!pip install numpy

     |████████████████████████████████| 65 kB 1.8 MB/s 
     |████████████████████████████████| 49 kB 3.8 MB/s 
     |████████████████████████████████| 45 kB 1.8 MB/s 
  Created wheel for prophet: filename=prophet-1.0.1-py3-none-any.whl size=6639580 sha256=e914932f4b8bff6a733939f055b583b693eb98f8e38a36a7f8b2ab05f37f4c50
  Stored in directory: /root/.cache/pip/wheels/4e/a0/1a/02c9ec9e3e9de6bdbb3d769d11992a6926889d71567d6b9b67
Successfully built prophet
  Attempting uninstall: cmdstanpy
    Found existing installation: cmdstanpy 0.9.5
    Uninstalling cmdstanpy-0.9.5:
      Successfully uninstalled cmdstanpy-0.9.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
fbprophet 0.7.1 requires cmdstanpy==0.9.5, but you have cmdstanpy 0.9.68 which is incompatible.
     |████████████████████████████████| 6.4 MB 16.7 MB/s 
     |████████████████████████████████| 63 kB 1.2 M

In [2]:
# Libraries
from tqdm import tqdm
import pandas as pd
from prophet import Prophet
import yfinance as yf
from datetime import datetime, timedelta
import plotly.express as px
import numpy as np

In [3]:
# Time periods
now = datetime.now()

# How far back to retrieve
ago = now - timedelta(days=730)

# Designating the Ticker
crypto = yf.Ticker("ETH-USD")

# Getting price history
df = crypto.history(start=ago.strftime("%Y-%m-%d"), end=now.strftime("%Y-%m-%d"), interval="1d")

# Handling missing data from yahoo finance
df = df.reindex(
    [df.index.min()+pd.offsets.Day(i) for i in range(df.shape[0])],
    fill_value=None
).fillna(method='ffill')

In [4]:
# Getting the N Day Moving Average and rounding the values
df['MA'] = df[['Open']].rolling(window=5).mean().apply(lambda x: round(x, 2))

# Dropping the NaNs
df.dropna(inplace=True)

In [5]:
# Formatted
df = df.reset_index().rename(columns={"Date": "ds", "MA": "y"})
df

,ds,Open,High,Low,Close,Volume,Dividends,Stock Splits,y
0,2020-04-21,172.017715,175.178467,170.552841,172.737701,16458767984,0,0,176.85
1,2020-04-22,172.670395,183.751007,171.826599,182.599579,17994666395,0,0,176.93
2,2020-04-23,182.620178,189.088867,180.340652,185.028671,21275740032,0,0,179.13
3,2020-04-24,185.222107,190.210388,185.222107,189.236938,16788555028,0,0,178.80
4,2020-04-25,189.207397,196.792374,188.459534,195.515305,18260969748,0,0,180.35
...,...,...,...,...,...,...,...,...,...
722,2022-04-13,3029.877930,3123.575439,3006.481689,3118.344238,16088473629,0,0,3134.83
723,2022-04-14,3117.821777,3139.928711,2988.435303,3019.909424,15821419931,0,0,3120.00
724,2022-04-15,3020.134521,3044.287354,3001.123291,3040.916504,11256651536,0,0,3071.77
725,2022-04-16,3041.123291,3074.886963,3016.421143,3062.310303,8766710365,0,0,3038.08


In [6]:
# Setting up Prophet
m = Prophet(
    daily_seasonality=True, 
    yearly_seasonality=True, 
    weekly_seasonality=True
)

# Fitting to the prices
m.fit(df[['ds', 'y']])

In [7]:
# Future DF of 3 months
future = m.make_future_dataframe(periods=90)

In [8]:
forecast = m.predict(future)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]

,ds,yhat,yhat_lower,yhat_upper
0,2020-04-21,-261.731811,-536.489980,19.007882
1,2020-04-22,-218.096721,-510.575604,67.472096
2,2020-04-23,-166.968423,-453.779952,130.206134
3,2020-04-24,-117.077230,-379.405186,160.404980
4,2020-04-25,-68.573206,-362.479159,199.890607
...,...,...,...,...
812,2022-07-12,1700.013753,1233.411730,2175.063840
813,2022-07-13,1680.486972,1203.100499,2133.273815
814,2022-07-14,1667.555301,1166.514185,2107.718950
815,2022-07-15,1652.864704,1172.250617,2117.568723


In [9]:
# Visual DF
vis_df = df[['ds','Open']].append(
    forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]
).rename(columns={'yhat': 'Prediction'})

# Visualizing results
fig = px.line(
    vis_df,
    x='ds',
    y=['Open', 'Prediction', 'yhat_lower', 'yhat_upper'],
    title='Crypto Forecast',
    labels={'value':'Price',
            'ds': 'Date'}
)

# Adding a slider
fig.update_xaxes(
    rangeselector=dict(
        buttons=list([
            dict(count=1, label="1m", step="month", stepmode="backward"),
            dict(count=3, label="3m", step="month", stepmode="backward"),
            dict(count=6, label="6m", step="month", stepmode="backward"),
            dict(count=1, label="YTD", step="year", stepmode="todate"),
            dict(count=1, label="1y", step="year", stepmode="backward"),
            dict(step="all")
        ])
    )
)

fig.show()

In [10]:
def getData(ticker, window, ma_period):
    """
    Grabs price data from a given ticker. Retrieves prices based on the given time window; from now
    to N days ago.  Sets the moving average period for prediction. Returns a preprocessed DF
    formatted for FB Prophet.
    """
    # Time periods
    now = datetime.now()

    # How far back to retrieve tweets
    ago = now - timedelta(days=window)

    # Designating the Ticker
    crypto = yf.Ticker(ticker)

    # Getting price history
    df = crypto.history(start=ago.strftime("%Y-%m-%d"), end=now.strftime("%Y-%m-%d"), interval="1d")
    
    # Handling missing data from yahoo finance
    df = df.reindex(
        [df.index.min()+pd.offsets.Day(i) for i in range(df.shape[0])],
        fill_value=None
    ).fillna(method='ffill')
    
    # Getting the N Day Moving Average and rounding the values
    df['MA'] = df[['Open']].rolling(window=ma_period).mean().apply(lambda x: round(x, 2))

    # Dropping the NaNs
    df.dropna(inplace=True)

    # Formatted for FB Prophet
    df = df.reset_index().rename(columns={"Date": "ds", "MA": "y"})
    
    return df

In [11]:
def fbpTrainPredict(df, forecast_period):
    """
    Uses FB Prophet and fits to a appropriately formatted DF. Makes a prediction N days into 
    the future based on given forecast period. Returns predicted values as a DF.
    """
    # Setting up prophet
    m = Prophet(
        daily_seasonality=True, 
        yearly_seasonality=True, 
        weekly_seasonality=True
    )
    
    # Fitting to the prices
    m.fit(df[['ds', 'y']])
    
    # Future DF
    future = m.make_future_dataframe(periods=forecast_period)
        
    # Predicting values
    forecast = m.predict(future)

    # Returning a set of predicted values
    return forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]

In [12]:
def visFBP(df, forecast):
    """
    Given two dataframes: before training df and a forecast df, returns
    a visual chart of the predicted values and actual values.
    """
    # Visual DF
    vis_df = df[['ds','Open']].append(forecast).rename(
        columns={'yhat': 'Prediction',
                 'yhat_upper': "Predicted High",
                 'yhat_lower': "Predicted Low"}
    )

    # Visualizing results
    fig = px.line(
        vis_df,
        x='ds',
        y=['Open', 'Prediction', 'Predicted High', 'Predicted Low'],
        title='Crypto Forecast',
        labels={'value':'Price',
                'ds': 'Date'}
    )

    # Adding a slider
    fig.update_xaxes(
        rangeselector=dict(
            buttons=list([
                dict(count=1, label="1m", step="month", stepmode="backward"),
                dict(count=3, label="3m", step="month", stepmode="backward"),
                dict(count=6, label="6m", step="month", stepmode="backward"),
                dict(count=1, label="YTD", step="year", stepmode="todate"),
                dict(count=1, label="1y", step="year", stepmode="backward"),
                dict(step="all")
            ])
        )
    )

    return fig.show()

In [13]:
# Getting and Formatting Data
df = getData("ETH-USD", window=365, ma_period=3)

# Training and Predicting Data
forecast = fbpTrainPredict(df, forecast_period=21)

# Visualizing Data
visFBP(df, forecast)

In [14]:
def runningFBP(ticker, window=730, ma_period=5, days_to_train=365, forecast_period=10):
    """
    Runs the facebook prophet model over the provided ticker.  Trains with last N days given 
    by days_to_train.  Forecast N days into the future based on given forecast_period.  Moving average 
    is applied to the dataset based on given ma_period. Returns the root mean squared error and a DF 
    of the actual values and the predicted values for the same day.
    """

    # Getting and Formatting Data
    df = getData(ticker, window=window, ma_period=ma_period)
    
    # DF for the predicted values
    pred_df = pd.DataFrame()

    # Running the model on each day
    for i in tqdm(range(days_to_train, window-forecast_period, forecast_period)):

        # Training and Predicting the last day on the forecast
        forecast = fbpTrainPredict(df[i-days_to_train:i], 
                                   forecast_period=forecast_period).tail(forecast_period)[['ds',
                                                                                           'yhat',
                                                                                           'yhat_lower',
                                                                                           'yhat_upper']]

        # Adding the last day predicted
        pred_df = pred_df.append(forecast, ignore_index=True)
        
    # Combining the predicted df and original df
    comb_df = df[['ds', 'Open']].merge(pred_df, 
                                       on='ds', 
                                       how='outer').sort_values(by='ds')
    
    # Setting the index to the dates
    comb_df.set_index('ds', inplace=True)

    return comb_df

In [15]:
def get_prophet_positions(df, short=True):
    """
    For these positions, buy when actual value is above the upper bound and short 
    when actual value is below lower bound. Otherwise do nothing.
    """
    if df['Open'] >= df['yhat_upper']:
        return 1
    elif df['Open'] <= df['yhat_lower'] and short:
        return -1
    else:
        return 0

In [16]:
def fbpBacktest(df, short=True):
    """
    Performs the final backtest using log returns and the positions function.
    Returns the performance.
    """
    # Getting positions
    df['positions'] = df.apply(lambda x: get_prophet_positions(x, short=short), axis=1)

    # Compensating for lookahead bias
    df['positions'] = df['positions'].shift(1)
    
    # Getting log returns
    df['log_returns'] = df['Open'].apply(np.log).diff()

    # Dropping any Nans
    df.dropna(inplace=True)
    
    # Performing the backtest
    returns = df['positions'] * df['log_returns']

    # Inversing the log returns and getting daily portfolio balance
    performance = returns.cumsum().apply(np.exp)
    
    return performance

In [17]:
# Running the model and getting forecast
bt_df = runningFBP("ETH-USD", 
                   window=730, 
                   ma_period=5, 
                   days_to_train=370, 
                   forecast_period=10)

# Performing the backtest
performance = fbpBacktest(bt_df, short=False)

# Visualizing results
px.line(performance,
        x=performance.index,
        y=performance,
        title='Portfolio Performance',
        labels={"y": "Portfolio Balance",
                "ds": "Date"})

100%|██████████| 35/35 [01:20<00:00,  2.30s/it]


In [18]:
bt_df


,Open,yhat,yhat_lower,yhat_upper,positions,log_returns
ds,,,,,,
2021-04-26,2319.478027,2418.817960,2343.304683,2493.162913,0.0,0.046354
2021-04-27,2534.031250,2430.289682,2356.460789,2499.558039,0.0,0.088469
2021-04-28,2664.685547,2439.267259,2365.526886,2510.877252,1.0,0.050275
2021-04-29,2748.649658,2446.381515,2373.200254,2524.806442,1.0,0.031024
2021-04-30,2757.734131,2450.762737,2373.882905,2526.738567,1.0,0.003300
...,...,...,...,...,...,...
2022-04-06,3411.672119,3468.319894,3258.332580,3683.439650,0.0,-0.031611
2022-04-07,3172.197266,3490.633120,3269.552754,3686.197454,0.0,-0.072778
2022-04-08,3233.272461,3506.954731,3279.217584,3726.006805,0.0,0.019070
